# (07) Align (within) -- lasso save (```GABAN```)

**Motivation**: Alignment scores (using lasso) within a model classes but different seeds. Generate and save results. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-MTMST/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-MTMST/figs')
tmp_dir = os.path.join(git_dir, 'jb-MTMST/tmp')

# Code path
sys.path.insert(0, '/home/hadi/Documents/MTMST/code')
from vae.train_vae import TrainerVAE, ConfigTrainVAE
from vae.vae2d import VAE, ConfigVAE
from figures.fighelper import *
from analysis.glm import *


# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm.notebook import tqdm
from rich.jupyter import print
%matplotlib inline
set_style()

In [2]:
def _fit_lasso(
    x,
    y,
    x_tst,
    y_tst,
    alpha=0.02, ):
    
    shape = (y.shape[1], x.shape[1])
    importances = np.zeros(shape)
    
    mu_x = x.mean(0, keepdims=True)
    sd_x = x.std(0, keepdims=True)
    x = normalize_global(x, mu_x, sd_x)

    mu_y = y.mean(0, keepdims=True)
    sd_y = y.std(0, keepdims=True)
    y = normalize_global(y, mu_y, sd_y)

    lasso = sk_linear.Lasso(alpha=alpha).fit(x, y)
    pred = lasso.predict(normalize_global(x_tst, mu_x, sd_x))
    
    r2 = sk_metric.r2_score(
        y_pred=pred,
        y_true=normalize_global(y_tst, mu_y, sd_y),
        multioutput='raw_values',
    )
    return lasso, r2


def _load_from_trainer(f: str):
    if not f.endswith('.npy'):
        f = f"{f}.npy"
    return np.load(
        file=pjoin(path_tr, f),
        allow_pickle=True,
    ).item()


def _fit_loop(
    seed_info: dict,
    mod: str,
    beta: float,
    device: str = 'cuda:1', ):
    
    # load data
    data = {}
    for seed, info in seed_info.items():
        if isinstance(info, str):
            d = _load_from_trainer(info)
            z = {
                k: d[f'data_{k}']['z']
                for k in ['vld', 'tst']
            }
        elif isinstance(info, tuple):
            model_name, fit_name = info
            tr, meta = load_model(
                model_name=model_name,
                fit_name=fit_name,
                device=device,
            )
            if not meta['checkpoint'] == 160:
                print(f"Warning:\nfit: {fit_name}\nchkpt = {meta['checkpoint']}")
            z = {
                k: tr.forward(k, True)[0]['z']
                for k in ['vld', 'tst']
            }
        else:
            raise NotImplementedError
        data[seed] = z
    
    print('[PROGRESS] loading data: done')
    
    # start fit loop
    for s1, s2 in itertools.combinations(seed_info, 2):
        for src, tgt in [(s1, s2), (s2, s1)]:
            fname = '_'.join([
                f"lasso-{mod}",
                f"beta-{beta}",
                f"src-{src}",
                f"tgt-{tgt}",
            ])
            if os.path.isfile(pjoin(save_dir, f"{fname}.npy")):
                print(f"fit '{fname}' already exists, skipping . . .")
                continue

            x_vld, x_tst = data[src]['vld'].copy(), data[src]['tst'].copy()
            y_vld, y_tst = data[tgt]['vld'].copy(), data[tgt]['tst'].copy()

            lasso, r2 = _fit_lasso(
                x=x_vld,
                y=y_vld,
                x_tst=x_tst,
                y_tst=y_tst,
                alpha=0.02,
            )
            w = np.abs(lasso.coef_)
            w /= w.sum(1, keepdims=True)
            a = 1 - entropy_normalized(w, axis=1)

            results = {
                'model': mod,
                'beta': beta,
                'cond': f"s{src} -> s{tgt}",
                'w': w,
                'a': a,
                'r2': r2,
            }
            save_obj(
                obj=results,
                file_name=fname,
                save_dir=save_dir,
                verbose=True,
                mode='npy',
            )
    return

## Setup

In [3]:
category = 'fixate1'
pal_model, pal_cat = get_palette()

path_tr = pjoin(tmp_dir, 'trainer_analysis')
save_dir = pjoin(tmp_dir, 'alignment_rebuttal')
print(save_dir)

/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

## VAE, $\beta = 1.0$

In [4]:
seed_info = {
    0: 'vanilla_fixate1_nf-420_beta-1.0_(2023_05_07,01:18)',
    1: (
        'fixate1_k-40_z-420x[1]_enc(0x3)-dec(0x2)-pre(3x13)-post(3x12)',
        'v1_ep160-b600-lr(0.002)_beta(1:0x0.5)_lamb(0.0001)_gr(250.0)_(2023_08_05,10:20)'),
    2: (
        'fixate1_k-40_z-420x[1]_enc(0x3)-dec(0x2)-pre(3x13)-post(3x12)',
        'v2_ep160-b600-lr(0.002)_beta(1:0x0.5)_lamb(0.0001)_gr(250.0)_(2023_08_05,10:21)'),
    3: (
        'fixate1_k-40_z-420x[1]_enc(0x3)-dec(0x2)-pre(3x13)-post(3x12)',
        'v3_ep160-b600-lr(0.002)_beta(1:0x0.5)_lamb(0.0001)_gr(250.0)_(2023_08_05,10:21)'),
}

In [5]:
%%time

_fit_loop(seed_info, 'VAE', 1.0, 'cuda')

[PROGRESS] loading data: done

[PROGRESS] 'lasso-VAE_beta-1.0_src-0_tgt-1.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

[PROGRESS] 'lasso-VAE_beta-1.0_src-1_tgt-0.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

[PROGRESS] 'lasso-VAE_beta-1.0_src-0_tgt-2.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

[PROGRESS] 'lasso-VAE_beta-1.0_src-2_tgt-0.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

...

[PROGRESS] 'lasso-VAE_beta-1.0_src-1_tgt-3.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

[PROGRESS] 'lasso-VAE_beta-1.0_src-3_tgt-1.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

[PROGRESS] 'lasso-VAE_beta-1.0_src-2_tgt-3.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

[PROGRESS] 'lasso-VAE_beta-1.0_src-3_tgt-2.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

CPU times: user 21h 4min 21s, sys: 7min 8s, total: 21h 11min 30s
Wall time: 1h 14min 55s


## cNVAE, $\beta = 1.0$

In [4]:
seed_info = {
    0: 'fixate1_nf-420_beta-1.0_(2023_04_29,19:41)',
    1: (
        'fixate1_k-32_z-20x[3,6,12]_enc(1x3)-dec(1x2)-pre(1x3)-post(1x3)',
        's1_ep160-b600-lr(0.002)_beta(1:0x0.5)_lamb(0.0001)_gr(250.0)_(2023_08_03,22:56)'),
    2: (
        'fixate1_k-32_z-20x[3,6,12]_enc(1x3)-dec(1x2)-pre(1x3)-post(1x3)',
        's2_ep160-b600-lr(0.002)_beta(1:0x0.5)_lamb(0.0001)_gr(250.0)_(2023_08_03,22:56)'),
    3: (
        'fixate1_k-32_z-20x[3,6,12]_enc(1x3)-dec(1x2)-pre(1x3)-post(1x3)',
        's3_ep160-b600-lr(0.002)_beta(1:0x0.5)_lamb(0.0001)_gr(250.0)_(2023_08_03,22:56)'),
}

In [5]:
%%time

_fit_loop(seed_info, 'cNVAE', 1.0, 'cuda')

[PROGRESS] loading data: done

fit 'lasso-cNVAE_beta-1.0_src-0_tgt-1' already exists, skipping . . .

fit 'lasso-cNVAE_beta-1.0_src-1_tgt-0' already exists, skipping . . .

/tmp/ipykernel_4136915/2231843439.py:97: RuntimeWarning: invalid value encountered in true_divide
  w /= w.sum(1, keepdims=True)


[PROGRESS] 'lasso-cNVAE_beta-1.0_src-0_tgt-2.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

...

/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.656e+01, tolerance: 7.337e+00
  model = cd_fast.enet_coordinate_descent(
/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.984e+00, tolerance: 7.386e+00
  model = cd_fast.enet_coordinate_descent(
/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8

/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.636e+00, tolerance: 7.500e+00
  model = cd_fast.enet_coordinate_descent(
/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.373e+01, tolerance: 7.500e+00
  model = cd_fast.enet_coordinate_descent(
/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2

/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.263e+01, tolerance: 7.500e+00
  model = cd_fast.enet_coordinate_descent(
/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.321e+01, tolerance: 7.500e+00
  model = cd_fast.enet_coordinate_descent(
/home/hadi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1

[PROGRESS] 'lasso-cNVAE_beta-1.0_src-3_tgt-2.npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/alignment_rebuttal

CPU times: user 2d 14h 40min 57s, sys: 11min 36s, total: 2d 14h 52min 33s
Wall time: 3h 25min 59s
